# Building the NLP model:

Now that we completed our EDA and checked our finindings using statistical tests, we can move on to our main task: Build a NLP model that takes in a video title as an input and predicts the engagement rate of the video.